In [ ]:
%load_ext autoreload
%autoreload 2

Declaration of parameters (you must also add a tag for this cell - parameters)

In [ ]:
# specify substep parameters for interactive run
# this cell will be replaced during job run with the parameters from json within params subfolder
substep_params={
    'save_best': False
}

In [ ]:
# load pipeline and step parameters - do not edit
from sinara.substep import get_pipeline_params, get_step_params
pipeline_params = get_pipeline_params(pprint=True)
step_params = get_step_params(pprint=True)

In [ ]:
# define substep interface
from sinara.substep import NotebookSubstep, ENV_NAME, PIPELINE_NAME, ZONE_NAME, STEP_NAME, RUN_ID, ENTITY_NAME, ENTITY_PATH, SUBSTEP_NAME

substep = NotebookSubstep(pipeline_params, step_params, substep_params)

substep.interface(
    inputs =
    [ 
      { STEP_NAME: "model_train", ENTITY_NAME: "obj_detect_inference_files"} # stored detector files from train step
    ],
    
    tmp_entities =
    [
        { ENTITY_NAME: "obj_detect_inference_files" } # temporary detector files from train step
    ],
    
    outputs =
    [
        { ENTITY_NAME: "bento_service" } # stored BentoService
    ],
)

substep.print_interface_info()

substep.exit_in_visualize_mode()

In [ ]:
# specify all notebook wide libraries imports here
# Sinara lib imports is left in the place of their usage
import json
import os
import os.path as osp
import glob
import torch

In [ ]:
# run spark
from sinara.spark import SinaraSpark
from sinara.archive import SinaraArchive

spark = SinaraSpark.run_session(0)
archive = SinaraArchive(spark)
SinaraSpark.ui_url()

### Loading obj_detect_inference_files from the model_train step 
(weights, configs, test image)

In [ ]:
inputs = substep.inputs(step_name = "model_train")
tmp_entities = substep.tmp_entities()
# copy config from previos step to outputs

archive.unpack_files_from_store_to_tmp(store_path=inputs.obj_detect_inference_files, tmp_entity_dir=tmp_entities.obj_detect_inference_files)

### Select obj_detector weights and config for packaging

In [ ]:
# Selecting a weights file to convert to onnx format (best, latest epoch, etc.)

best_weights_pths = glob.glob(f"{tmp_entities.obj_detect_inference_files}/*best*")
latest_weights_pths = glob.glob(f"{tmp_entities.obj_detect_inference_files}/*latest*")

weights_pths = best_weights_pths if substep_params['save_best'] and len(best_weights_pths) > 0 else latest_weights_pths

# Select weigths file with max filetime
weights_pths.sort(key=lambda file: osp.getmtime(file))
selected_weights_pth = weights_pths[-1]

# Clean model weights - remove optimizer's data
state_dict = torch.load(selected_weights_pth, map_location=torch.device('cpu'))
print(f"{state_dict.keys()=}")
if "optimizer" in state_dict:
    del state_dict["optimizer"]

# Prepare files for packaging
clean_weigths_pth = osp.splitext(selected_weights_pth)[0]+"_clean.pth"
torch.save(state_dict, clean_weigths_pth)

mmengine_cfg = osp.join(tmp_entities.obj_detect_inference_files, "last_cfg.py")

### Pack binary  model

#### Packaging of obj_detector in bento_service

In [ ]:
from sinara.bentoml.bento_archive import BentoArchive

# initialize bento service
model_service = BentoArchive()

# get model weights as binary file for bento_artifact
with open(clean_weigths_pth, "rb") as f_id:
    clean_weigths_pth_binary = f_id.read()
    
# get model config as binary file for bento_artifact
with open(mmengine_cfg, "rb") as f_id:
    mmengine_cfg_binary = f_id.read()
    
model_service.pack('weigths_pth', clean_weigths_pth_binary, file_extension='pth')
model_service.pack('mmengine_cfg', mmengine_cfg_binary, file_extension='py')

test_image_path = osp.join(tmp_entities.obj_detect_inference_files, "test.jpg")
assert osp.exists(test_image_path)

# get test image as binary file for bento_artifact
with open(test_image_path, "rb") as f_id:
    f_test_image = f_id.read()
    
model_service.pack('test_image', f_test_image, file_extension='jpg')

### Send packaged obj_detector to outputs

In [ ]:
# save model as a bento pack
from sinara.bentoml import save_bentoservice

outputs = substep.outputs()

save_bentoservice(model_service, path=outputs.bento_service, substep=substep)

In [ ]:
# stop spark
SinaraSpark.stop_session()